In [19]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

C:\Users\scanimage\AppData\Local\Temp\ipykernel_8464\912229180.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [20]:
import sys
import os
import glob
import importlib
import multiprocessing as mp
import itertools
import math
import random 

from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from matplotlib import colors as mcolors
import scipy.sparse as sparse
import scipy.io as sio
import scipy.stats as stats
import scipy.spatial.distance as dist
import numpy as np
import h5py
from tqdm import tqdm
import pandas as pd
#import tables
#import deepdish as dd

sys.path.append('C:\\Users\\scanimage\\Documents\\JJM\\post_cnmfe_analysis\\')
import python_utils_jjm as jjm_utils

In [22]:
os.listdir('F:\\JJM\\miniscope_analysis_CA1\\cnmfe_output\\14_26_45_21928_run00\\')

['13-Nov_18_04_16_out.mat',
 'new_sessions_2023_09_28_My_V4_Miniscope_com_filtered.csv',
 'new_sessions_2023_09_28_My_V4_Miniscope_C_traces_filtered.csv']

In [24]:
results_file = 'F:\\JJM\\miniscope_analysis_CA1\\cnmfe_output\\14_26_45_21928_run00\\13-Nov_18_04_16_out.mat'
results = sio.loadmat(results_file)

In [25]:
results['A']

<360000x408 sparse matrix of type '<class 'numpy.float64'>'
	with 196138 stored elements in Compressed Sparse Column format>

##plot contours from A, spatial profile of detected neurons, on correlation image (Cn) or image from file 

In [26]:
#input list to plot
to_plot = [1, 3, 5, 6, 12, 13]

In [27]:
# A is stored as "sparse matrix", convert to dense 
sparse.issparse(results['A'])
A = np.array(results['A'].todense())

In [28]:
img_file = 'F:\\JJM\\miniscope_analysis_CA1\\figure\\21928_MAX_Substack_500_1000-1.tif'
img_from_file = mpimg.imread(img_file) 
#plt.imshow(img_from_file)

In [29]:
np.shape(img_from_file)

(600, 600)

In [30]:
# Correlation image 
#Cn = results['Cn']
Cn = np.transpose(img_from_file)
#c, d1, d2 = np.shape(Cn)
d1, d2 = np.shape(Cn)
d, nr = np.shape(A)

In [31]:
# threshold for contour
maxthr=0.4
nrgthr=0.9

In [32]:
# mesh grid that represents the x, y dimensions of the image
x, y = np.mgrid[0:d1:1, 0:d2:1]

In [33]:
#create plot
#%matplotlib inline
#ax = plt.gca()
#cmap=None
#plt.imshow(np.transpose(Cn), interpolation=None, cmap='gray', vmin=np.percentile(Cn[~np.isnan(Cn)], 1),vmax=np.percentile(Cn[~np.isnan(Cn)], 99))

In [34]:
coordinates = []
#cm is an array of tuples 
cm = jjm_utils.com(A, d1, d2)

In [35]:
len(results['C'])

408

In [36]:
#[int(i) for i in np.linspace(1,126,126)]

In [37]:
to_plot = [1, 3, 5, 6, 12, 13, 14, 20, 24, 44, 51]
#to_plot = [int(i) for i in np.linspace(0,220,50)]
#to_plot

In [18]:
list(mcolors.TABLEAU_COLORS.values())

['#1f77b4',
 '#ff7f0e',
 '#2ca02c',
 '#d62728',
 '#9467bd',
 '#8c564b',
 '#e377c2',
 '#7f7f7f',
 '#bcbd22',
 '#17becf']

In [ ]:
com_cnmfe = pd.read_csv('F:\JJM\miniscope_analysis\dSPNs\clustering_analysis\DIO_r2_19_Day_3_13_54_25_com_filtered.csv')

In [ ]:
[int(i) for i in list(com_cnmfe.columns)[2:]]

In [ ]:
#rotated_image = np.rot90(img_from_file, k=-1)
#rotated_image = np.rot90(rotated_image, k=-1)
#rotated_image = np.rot90(rotated_image, k=-1)
to_plot = [3, 
           5, 6, 13, 14, 20, 24, 44, 51, 55, 65, 95, 105, 135, 145, 155,
          0, 10, 20, 30, 40, 50, 60, 70, 80, 100, 110, 140, 150, 180, 190, 210, 220,
         113, 123, 133,
          19, 25, 30, 31, 32, 35, 43, 47, 48]
#           25, 27, 28, 30, 31, 32, 34, 35, 43, 47, 48, 49, 55, 62]

maxthr = 0.4
fig, ax = plt.subplots()

ax.imshow(img_from_file, cmap='gray')

ax.axis('off')

if to_plot:
    cells_to_plot = to_plot

cells_to_plot

if to_plot:
    cells_to_plot = to_plot
else:
    max_number=nr
    cells_to_plot = range(np.minimum(nr, max_number))

contour_colors = [random.choice(list(mcolors.TABLEAU_COLORS.values())) for _ in range(len(cells_to_plot))]    
    
for i, color in zip(cells_to_plot, contour_colors):
    #pars = dict(kwargs)
    Bvec = A[:, i].flatten()
    #normalize spatial contours to 1 
    Bvec /= np.max(Bvec)
    thr = maxthr
# reshape to dimensions of image 
    Bmat = np.reshape(Bvec, (d1, d2), order='F')
    colors = 'r'
    cs = plt.contour(y, x, Bmat, [thr], colors=color)
    #cs = plt.contour(y, x, Bmat, [thr], color='r')
    for contour in cs.collections:
        paths = contour.get_paths()
        for path in paths:
            x_center = np.mean(path.vertices[:, 0])
            y_center = np.mean(path.vertices[:, 1])

            # Add text annotation with the cell index at the center of the contour
#            plt.text(x_center, y_center, str(i), color='r', fontsize=6)
plt.savefig('F:\\JJM\\Marshall_et_al_2023\\Figure1\\eg_image\\DIO_r2_19_14_20_58\\imgwithcontours.tif')
#plt.savefig('F:\\JJM\\Marshall_et_al_2023\\Figure1\\eg_image\\DIO_r2_19_14_20_58\\contours.tif', format='svg', transparent=True)

In [ ]:
region_to_plot = (0,1000)
cells_to_plot = [1, 5, 7, 8, 14, 19, 20, 29]
N = len(cells_to_plot)
fig, axs = plt.subplots(N, 1, figsize=(6, 2 * N), sharey=True) 

for ax in axs:
    ax.axis('off')

for i, color, ax in zip(cells_to_plot, contour_colors, axs):
    ax.plot(results['C'][i][region_to_plot[0]:region_to_plot[1]], color=color)
    
plt.tight_layout()
plt.savefig('F:\\JJM\\Marshall_et_al_2023\\Figure1\\eg_image\\DIO_r2_19_14_20_58\\traces.svg')
plt.show()

In [ ]:
#### plot one with scale
region_to_plot = (0,1000)
to_plot_scale =[1, 5, 7, 8, 14, 19, 20, 29]
N = len(to_plot_scale)
fig, axs = plt.subplots(N, 1, figsize=(6, 2 * N), sharey=True) 

for ax in axs:
    ax.axis('on')

for i, color, ax in zip(to_plot_scale, contour_colors, axs):
    ax.plot(results['C'][i][region_to_plot[0]:region_to_plot[1]], color=color)
    
plt.tight_layout()
plt.savefig('F:\\JJM\\Marshall_et_al_2023\\Figure1\\eg_image\\DIO_r2_19_14_20_58\\traces_scale.tif')
plt.show()

In [ ]:
len(results['C'][1])

In [ ]:
coordinates = []
cm = jjm_utils.com(A, d1, d2)

if to_plot:
    cells_to_plot = to_plot

cells_to_plot

if to_plot:
    cells_to_plot = to_plot
else:
    max_number=nr
    cells_to_plot = range(np.minimum(nr, max_number))

for i in cells_to_plot:
    #pars = dict(kwargs)
    Bvec = A[:, i].flatten()
    #normalize spatial contours to 1 
    Bvec /= np.max(Bvec)
    thr = maxthr
# reshape to dimensions of image 
    Bmat = np.reshape(Bvec, np.shape(Cn), order='F')
    colors = 'r'
    cs = plt.contour(y, x, Bmat, [thr], colors=colors)
    plt.savefig('F:\\JJM\\miniscope_analysis\\figures\\GRIN011_egforfigure\\MAX_GRIN011_H10_M19_S59msCam9_390_440_w_contours.png')

In [ ]:
plt.savefig('F:\\JJM\\miniscope_analysis\\figures\\GRIN011_egforfigure\\MAX_GRIN011_H10_M19_S59msCam9_390_440_w_contours.png')

In [ ]:
cs